## Filling Demand while Optimizing Cost

In cells 1-6, we import packages and read in the files we'll use. Refer to the assignment instructions for a description of the files. Note that we drop any null columns/rows.

In [ ]:
import pandas as pd
import numpy as np
import pulp

In [ ]:
orders = pd.read_excel('Supply chain logisitcs problem.xlsx')
orders.dropna(axis = 1, how = 'all', inplace = True)
orders.dropna(axis = 0, how = 'all', inplace = True)
orders.head()

In [ ]:
freight_rates = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 1)
freight_rates.dropna(axis = 1, how = 'all', inplace = True)
freight_rates.dropna(axis = 0, how = 'all', inplace = True)
freight_rates.head()

In [ ]:
wh_capacities = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 2)
wh_capacities.dropna(axis = 1, how = 'all', inplace = True)
wh_capacities.dropna(axis = 0, how = 'all', inplace = True)
wh_capacities.head()

In [ ]:
products_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 3)
products_per_plant.dropna(axis = 1, how = 'all', inplace = True)
products_per_plant.dropna(axis = 0, how = 'all', inplace = True)
products_per_plant.head()

In [ ]:
ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 4)
ports.dropna(axis = 1, how = 'all', inplace = True)
ports.dropna(axis = 0, how = 'all', inplace = True)
ports.head()

We need to turn the shipping costs into a dictionary for easy lookup. We use the 'dict(zip(column1, column2))' paradigm.

In [ ]:
shipping_costs = dict(zip(freight_rates['orig_port_cd'], freight_rates['rate']))

Next, we create a list of all unique products per plant. For now, you can treat the 'tuple' data type as a list.

In [ ]:
def get_plants(product_id):
    
    temp = products_per_plant[products_per_plant['Product ID'] == product_id]
    return tuple(np.unique(temp['Plant Code']))

## Part 1 (20 points)

a) 10 points

b) 10 points

In [ ]:
''' 
a) Create a new column in the 'orders' dataframe called 'allowed_plants'. 
To do this, you'll need to apply the defined get_plants function using a lambda function.
'''

# your code here


In [ ]:
''' 
End of Section
'''

In [ ]:
'''
b) Set the index of the 'orders' dataframe to be the 'Order ID'. Make sure you set the index in place. 
'''
# your code here


In [ ]:
''' 
End of Section
'''

Next, we create a dictionary to connect plants (warehouses) with the associated ports. Again, we use the 'dict(zip(column1, column2))' paradigm.

In [ ]:
plant_ports = dict(zip(ports['Plant Code'], ports['Port']))

## Part 2 (60 points)

a) 25 points

b) 25 points

c) 10 points

In [ ]:
def production_cost(order_id, plant):
    '''
    a) Return the production cost for a given order_id and plant (wahrehouse) name. 
    From the order id, you should first get the associated product id, which can be used to get the cost per unit.
    From here, multiply the cost per unit by the unit quantity to get the total production cost.
    '''
    
    # your code here
    

In [ ]:
'''
End of Section
'''

In [ ]:
def shipping_cost(order_id, plant):
    '''
    b) Return the shipping cost for a given order_id and plant (warehouse) name. 
    From the plant name, you should first get the associated port, which can be used to get the shipping cost per lb.
    From here, multiply the cost per lb by the weight to get the total shipping cost.
    '''
    
    # your code here
    

In [ ]:
'''
End of Section
'''

In [ ]:
def total_cost(order_id, plant):
    '''
    c) Return the total cost for a given order_id and plant (warehouse) name. 
    You should add the results of the two functions above. 
    '''
    # your code here
    

In [ ]:
'''
End of Section
'''

In [ ]:
### We create a dictionary with the key-value pair 'orderId_plantName': total_cost.

order_costs = {}
for name, row in orders.iterrows():
    order_id = name
    for plant in row['allowed_plants']:   
        order_costs[str(order_id) + '_' + str(plant)] = total_cost(order_id, plant)

In [ ]:
### We create a dictionary with the key-value pair 'plantName': list_of_orders.

plants = np.unique(ports['Plant Code'])

plant_orders = {}
for plant in plants:
    temp_list = []
    for name, row in orders.iterrows():
        if plant in row['allowed_plants']:  
            temp_list.append(str(name) + '_' + plant)
    plant_orders[plant] = temp_list

In [ ]:
### We create a dictionary with the key-value pair 'plantName': capacity.

plant_cap = dict(zip(wh_capacities['Plant ID'], wh_capacities['Daily Capacity']))

In [ ]:
### We create a dictionary with the key-value pair 'orderID': orderID_plantName.

order_plants = {}
temp_dict = dict(zip(orders.index, orders['allowed_plants']))
for key in temp_dict:
    temp_list = []
    for pl in temp_dict[key]:
        temp_list.append(str(key) + '_' + pl)
    order_plants[key] = temp_list

### Creating linear programming constraints

In this section, we build the linear programming problem and solve.

In [ ]:
build = pulp.LpVariable.dicts("Route",order_costs.keys(),0,None, pulp.LpInteger)
prob = pulp.LpProblem("Problem",pulp.LpMinimize)
prob += pulp.lpSum([build[b] * order_costs[b] for b in order_costs.keys()]), "Total Cost"

In [ ]:
for plant in plant_orders:
    if len(plant_orders[plant]) > 0:
        prob += pulp.lpSum(build[p] for p in plant_orders[plant]) <= plant_cap[plant], "Total orders out of plant %s"%plant

In [ ]:
for o in order_plants:
    prob += pulp.lpSum(build[p] for p in order_plants[o]) == 1, "Order_" + str(o) + "_filled"

## Part 3 (20 points)
a) 10 pts

c) 10 pts

In [ ]:
''' 
a) Solve the linear programming problem and store its status in a variable called 'status'.
'''

# your code here


print("Status:", status)

In [ ]:
'''
End of Section
'''

In [ ]:
'''
b) Find the total cost to produce and ship all products and store the answer in a variable called total_cost
Round the final answer to 2 decimal places (https://docs.python.org/3/library/functions.html#round).
'''
# your code here


print("Total Cost = ", str(total_cost))

In [ ]:
'''
End of Section
'''